In [1]:
library(mice)
library(finalfit)
library(ggplot2)
library(dplyr)
library(stats)
library(miceadds)
library(missMethods)
library(smotefamily)
library(caret)
library(cowplot)
library(mlr)
library(fastDummies)
library(DMwR2)
library(mlmi)
library(ranger)
library(scales)
library(Hmisc)

Warning message:
"pakiet 'mice' został zbudowany w wersji R 4.3.2"

Dołączanie pakietu: 'mice'


Następujący obiekt został zakryty z 'package:stats':

    filter


Następujące obiekty zostały zakryte z 'package:base':

    cbind, rbind


Warning message:
"pakiet 'ggplot2' został zbudowany w wersji R 4.3.3"

Dołączanie pakietu: 'dplyr'


Następujące obiekty zostały zakryte z 'package:stats':

    filter, lag


Następujące obiekty zostały zakryte z 'package:base':

    intersect, setdiff, setequal, union


* miceadds 3.16-18 (2023-01-06 10:54:00)

Ładowanie wymaganego pakietu: lattice

Warning message:
"pakiet 'cowplot' został zbudowany w wersji R 4.3.2"
Warning message:
"pakiet 'mlr' został zbudowany w wersji R 4.3.2"
Ładowanie wymaganego pakietu: ParamHelpers

Warning message:
"pakiet 'ParamHelpers' został zbudowany w wersji R 4.3.2"
Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to t

In [2]:
create_zero_df <- function(num_rows, num_cols, colnames) {
  #' Create dataframe of zeros.
  
  df <- as.data.frame(matrix(0, nrow = num_rows, ncol = num_cols))
  names(df) <- colnames
  return(df)
}

In [3]:
test_impute_mice <- function(data, m, maxit, method, formula, family, intcols) {
    #' Compute statistical estimates for data imputed with mice package.

    if (m > 1) {
        imp <- mice(data, method=method, m=m, maxit=maxit, printFlag=FALSE)
        imp[intcols] <- imp[intcols] %>% lapply(as.integer)
        fit <- with(imp, glm(formula=as.formula(formula), family=family))
        tab <- summary(pool(fit), "all", conf.int = TRUE)
        tab <- tab[, c("term", "estimate", "2.5 %", "97.5 %")]
        tab <- as.data.frame(tab)
    }

    else {
        imp <- mice(data, method=method, m=m, maxit=maxit, printFlag=FALSE) %>% complete()
        imp[intcols] <- imp[intcols] %>% lapply(as.integer)
        fit <- glm(formula=formula, family=family, data=imp)

        coefs <- as.data.frame(fit$coefficients)
        colnames(coefs) <- c("estimate")
        coefs <- cbind(term=rownames(coefs), coefs)
        rownames(coefs) <- NULL

        conf <- confint(fit)
        rownames(conf) <- NULL
        
        tab <- cbind(coefs, conf)
    }
    
    return(tab)
}

In [4]:
simulate_mice <- function(nsim, data, m, maxit, method, formula, family, intcols, nvars) {
    #' Simulate statistical estimates computation for data imputed with mice package.

    nrows <- nvars + 1
    
    tst_imp <- create_zero_df(nrows*nsim, 4, c("term", "estimate", "2.5 %", "97.5 %"))

    for (n in 1:nsim) {
        data_n <- sample_n(data, size=nrow(data), replace=TRUE)
        start_index <- (n - 1) * nrows + 1
        end_index <- n * nrows
        tst_imp_n <- test_impute_mice(data_n, m, maxit, method, formula, family, intcols)
        tst_imp[start_index:end_index, ] <- tst_imp_n
    }

    return(tst_imp)
}

In [5]:
pool_mlmi_helper <- function(data, formula, family) {
    #' Helper function for pooling estimates after MLMI imputation.

    fit <- glm(formula=as.formula(formula), family=family, data=data)
    coefs <- as.data.frame(fit$coefficients)
    colnames(coefs) <- c("estimate")
    coefs <- cbind(term=rownames(coefs), coefs)
    rownames(coefs) <- NULL

    conf <- confint(fit)
    rownames(conf) <- NULL
    
    res <- cbind(coefs, conf)

    return(res)
}

In [6]:
pool_mlmi <- function(imp, M, formula, family) {
    #' Pool statistical estimates for data imputed with MLMI.

    data <- imp[[1]]
    res <- pool_mlmi_helper(data, formula, family)

    for (i in 2:M) {
        data_i <- imp[[i]]
        res_i <- pool_mlmi_helper(data_i, formula, family)
        res <- rbind(res, res_i)
    }

    pooled <- aggregate(res[, c(2, 3, 4)], list(res$term), FUN=mean)
    colnames(pooled)[1] <- "term"

    return(pooled)

}

In [7]:
test_impute_mlmi <- function(data, M, nCat, rseed, formula, family, bcols, fcols, intcols) {
    #' Compute statistical estimates for data imputed with MLMI.
    
    imp <- mixImp(obsData = data, nCat = nCat, M = M, rseed = rseed)
    
    for (i in 1:M) {

        for (bcol in bcols) {
            imp[[i]][bcol] <- ifelse(imp[[i]][bcol] == 1, 0, 1)
        }

        imp[[i]][fcols] <- imp[[i]][fcols] %>% lapply(factor)
        imp[[i]][intcols] <- imp[[i]][intcols] %>% lapply(as.integer)
    }
    
    pooled <- pool_mlmi(imp, M, formula, family)

    return(pooled)
}

In [8]:
simulate_mlmi <- function(nsim, data, M, nCat, formula, family, bcols, fcols, intcols, nvars) {
    #' Simulate statistical estimates computation for data imputed with MLMI.
    
    nrows <- nvars + 1
    
    tst_imp <- create_zero_df(nrows*nsim, 4, c("term", "estimate", "2.5 %", "97.5 %"))

    for (n in 1:nsim) {
        data_n <- sample_n(data, size=nrow(data), replace=TRUE)
        rseed <- sample(1:5000, 1)
        tst_imp_n <- test_impute_mlmi(data_n, M, nCat, rseed, formula, family, bcols, fcols, intcols)
        start_index <- (n - 1) * nrows + 1
        end_index <- n * nrows
        tst_imp[start_index:end_index, ] <- tst_imp_n
    }

    return(as.data.frame(tst_imp))
}

In [9]:
test_impute_knn <- function(data, formula, family, intcols) {
    #' Compute statistical estimates for data imputed with KNN.

    imp <- knnImputation(data, scale=FALSE)
    imp[intcols] <- imp[intcols] %>% lapply(as.integer)

    fit <- glm(formula=formula, family=family, data=imp)

    coefs <- as.data.frame(fit$coefficients)
    colnames(coefs) <- c("estimate")
    coefs <- cbind(term=rownames(coefs), coefs)
    rownames(coefs) <- NULL

    conf <- confint(fit)
    rownames(conf) <- NULL
    
    tab <- cbind(coefs, conf)

    return(tab)
}

In [10]:
simulate_knn <- function(nsim, data, formula, family, intcols, nvars) {
    #' Simulate statistical estimates computation for data imputed with KNN.

    nrows <- nvars + 1
    
    tst_imp <- create_zero_df(nrows*nsim, 4, c("term", "estimate", "2.5 %", "97.5 %"))

    for (n in 1:nsim) {
        data_n <- sample_n(data, size=nrow(data), replace=TRUE)
        tst_imp_n <- test_impute_knn(data_n, formula, family, intcols)
        start_index <- (n - 1) * nrows + 1
        end_index <- n * nrows
        tst_imp[start_index:end_index, ] <- tst_imp_n
    }

    return(as.data.frame(tst_imp))
    
}

In [11]:
PB <- function(sim_imp, real_coefs) {
    #' Compute percent bias.

    exp_est <- aggregate(sim_imp$estimate, list(sim_imp$term), FUN=mean)
    rownames(exp_est) <- exp_est$Group.1
    exp_est <- subset(exp_est, select=-1)

    terms <- unique(sim_imp$term)
    pbs <- rep(0, length(terms))

    for (t in 1:length(terms)) {
        pbs[t] <- 100*abs((exp_est[terms[t],] - as.numeric(real_coefs[terms[t]])) / as.numeric(real_coefs[terms[t]]))
    }
    
    return(pbs)
}

In [12]:
CR <- function(sim_imp, real_coefs) {
    #' Compute coverage rate.

    terms <- unique(sim_imp$term)
    crs <- rep(0, length(terms))

    for (t in 1:length(terms)) {
        tsubs <- subset(sim_imp, term==terms[t])
        crs[t] <- mean(tsubs["2.5 %"] < as.numeric(real_coefs[terms[t]]) & as.numeric(real_coefs[terms[t]]) < tsubs["97.5 %"])
    }

    return(crs)
}

In [13]:
AW <- function(sim_imp, real_coefs) {
    #' Compute average width of confidence intervals.

    terms <- unique(sim_imp$term)
    aws <- rep(0, length(terms))

    for (t in 1:length(terms)) {
        tsubs <- subset(sim_imp, term==terms[t])
        diff <- tsubs["97.5 %"] - tsubs["2.5 %"]
        aws[t] <- mean(diff[, 1])
    }

    return(aws)   
}

In [14]:
fram <- read.csv("Framingham.csv")
fram_mcar <- read.csv("fram_mcar_num.csv")

In [15]:
fcols <- c("male", "education", "currentSmoker", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes", "TenYearCHD")
bcols <- c("male", "currentSmoker", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes", "TenYearCHD")
intcols <- c("age", "cigsPerDay", "glucose", "totChol", "heartRate")
fram_mcar[fcols] <- fram_mcar[fcols] %>% lapply(factor)
fram[fcols] <- fram[fcols] %>% lapply(factor)

In [16]:
glm_fit <- glm(formula = TenYearCHD ~ sysBP + diaBP + heartRate + glucose, family = "binomial", data = fram)

In [17]:
real_coefs <- glm_fit$coefficients
real_coefs

(Intercept)       sysBP       diaBP   heartRate     glucose 
-5.12018446  0.02540749 -0.00518078 -0.00719847  0.01111416

##### Mean imputation

In [18]:
method1 <- rep("", ncol(fram_mcar))
method1[c(11, 12, 14, 15)] <- "mean"

In [25]:
sim_imp1 <- simulate_mice(1000, fram_mcar, 1, 1, method1, "TenYearCHD ~ sysBP + diaBP + heartRate + glucose", "binomial", intcols, 4)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [26]:
pb1 <- PB(sim_imp1, real_coefs)
pb1

[1]   3.083584  14.823165 127.288216  19.482152   1.386804

In [27]:
cr1 <- CR(sim_imp1, real_coefs)
cr1

[1] 0.932 0.760 0.828 0.948 0.923

In [28]:
aw1 <- AW(sim_imp1, real_coefs)
aw1

[1] 2.271976582 0.012234400 0.025897099 0.020390887 0.009060381

##### Linear regression imputation

In [29]:
method2 <- rep("", ncol(fram_mcar))
method2[c(11, 12, 14, 15)] <- "norm.predict"

In [30]:
sim_imp2 <- simulate_mice(1000, fram_mcar, 1, 1, method2, "TenYearCHD ~ sysBP + diaBP + heartRate + glucose", "binomial", intcols, 4)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [31]:
pb2 <- PB(sim_imp2, real_coefs)
pb2

[1]   8.406135  25.829963 280.898691  24.067692   2.524856

In [32]:
cr2 <- CR(sim_imp2, real_coefs)
cr2

[1] 0.836 0.628 0.559 0.882 0.874

In [33]:
aw2 <- AW(sim_imp2, real_coefs)
aw2

[1] 2.096815166 0.016836748 0.032695826 0.020685506 0.009117144

##### Stochastic regression imputation

In [34]:
method3 <- rep("", ncol(fram_mcar))
method3[c(11, 12, 14, 15)] <- "norm.nob"

In [35]:
sim_imp3 <- simulate_mice(1000, fram_mcar, 1, 1, method3, "TenYearCHD ~ sysBP + diaBP + heartRate + glucose", "binomial", intcols, 4)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [36]:
pb3 <- PB(sim_imp3, real_coefs)
pb3

[1]   4.903854   5.816297 105.753590   3.247915   1.736870

In [37]:
cr3 <- CR(sim_imp3, real_coefs)
cr3

[1] 0.882 0.872 0.799 0.900 0.911

In [38]:
aw3 <- AW(sim_imp3, real_coefs)
aw3

[1] 1.988678022 0.014643207 0.027858616 0.018841190 0.008553207

##### Predictive mean matching

In [39]:
method4 <- rep("", ncol(fram_mcar))
method4[c(11, 12, 14, 15)] <- "pmm"

In [40]:
sim_imp4 <- simulate_mice(1000, fram_mcar, 1, 1, method4, "TenYearCHD ~ sysBP + diaBP + heartRate + glucose", "binomial", intcols, 4)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [41]:
pb4 <- PB(sim_imp4, real_coefs)
pb4

[1]   6.297217   5.982593 111.405156   1.689451   5.893150

In [42]:
cr4 <- CR(sim_imp4, real_coefs)
cr4

[1] 0.865 0.857 0.776 0.902 0.859

In [43]:
aw4 <- AW(sim_imp4, real_coefs)
aw4

[1] 1.990158903 0.014198189 0.027173546 0.018875115 0.008487076

##### MICE with linear regression

In [44]:
method5 <- rep("", ncol(fram_mcar))
method5[c(11, 12, 14, 15)] <- "norm.predict"

In [45]:
sim_imp5 <- simulate_mice(1000, fram_mcar, 5, 5, method5, "TenYearCHD ~ sysBP + diaBP + heartRate + glucose", "binomial", intcols, 4)

Warning message:
"Number of logged events: 25"


In [46]:
pb5 <- PB(sim_imp5, real_coefs)
pb5

[1]   9.8979646  24.9996057 292.3544236  17.1538605   0.5741582

In [47]:
cr5 <- CR(sim_imp5, real_coefs)
cr5

[1] 0.808 0.655 0.563 0.885 0.881

In [48]:
aw5 <- AW(sim_imp5, real_coefs)
aw5

[1] 2.08411776 0.01744682 0.03395578 0.02070641 0.00901844

##### MICE with stochastic regression

In [49]:
method6 <- rep("", ncol(fram_mcar))
method6[c(11, 12, 14, 15)] <- "norm.nob"

In [50]:
sim_imp6 <- simulate_mice(1000, fram_mcar, 5, 5, method6, "TenYearCHD ~ sysBP + diaBP + heartRate + glucose", "binomial", intcols, 4)

Warning message:
"Number of logged events: 50"
Warning message:
"Number of logged events: 25"
Warning message:
"Number of logged events: 25"
Warning message:
"Number of logged events: 25"
Warning message:
"Number of logged events: 25"
Warning message:
"Number of logged events: 25"
Warning message:
"Number of logged events: 25"


In [51]:
pb6 <- PB(sim_imp6, real_coefs)
pb6

[1]   6.74330000  10.20386509 150.24115215   0.04157387   5.34500179

In [52]:
cr6 <- CR(sim_imp6, real_coefs)
cr6

[1] 0.891 0.895 0.819 0.943 0.916

In [53]:
aw6 <- AW(sim_imp6, real_coefs)
aw6

[1] 2.098008233 0.017342823 0.033734248 0.020959921 0.009004434

##### MICE with pmm

In [54]:
method7 <- rep("", ncol(fram_mcar))
method7[c(11, 12, 14, 15)] <- "pmm"

In [55]:
sim_imp7 <- simulate_mice(1000, fram_mcar, 5, 5, method7, "TenYearCHD ~ sysBP + diaBP + heartRate + glucose", "binomial", intcols, 4)

Warning message:
"Number of logged events: 50"
Warning message:
"Number of logged events: 25"


In [56]:
pb7 <- PB(sim_imp7, real_coefs)
pb7

[1]   7.122678  10.185672 162.040930   9.126086   6.916090

In [57]:
cr7 <- CR(sim_imp7, real_coefs)
cr7

[1] 0.900 0.903 0.833 0.944 0.916

In [58]:
aw7 <- AW(sim_imp7, real_coefs)
aw7

[1] 2.164588278 0.017963943 0.035824549 0.021631108 0.009586391

##### KNN imputation

In [59]:
sim_imp8 <- simulate_knn(1000, fram_mcar, "TenYearCHD ~ sysBP + diaBP + heartRate + glucose", "binomial", intcols, 4)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [60]:
pb8 <- PB(sim_imp8, real_coefs)
pb8

[1]  6.561666  1.792562 79.223039 29.917451  9.540972

In [61]:
cr8 <- CR(sim_imp8, real_coefs)
cr8

[1] 0.891 0.936 0.879 0.914 0.874

In [62]:
aw8 <- AW(sim_imp8, real_coefs)
aw8

[1] 2.042410530 0.014633235 0.028824362 0.019277544 0.008473436

##### MLMI

In [63]:
catfram <- fram_mcar[, (names(fram_mcar) %in% fcols)]
contfram <- fram_mcar[, !(names(fram_mcar) %in% colnames(catfram))]
fram_reorder <- cbind(catfram, contfram)
fram_reorder[] <- lapply(fram_reorder, function(x) if(is.factor(x)) as.integer(x) else x)

In [64]:
sim_imp9 <- simulate_mlmi(1000, fram_reorder, 5, 8, "TenYearCHD ~ sysBP + diaBP + heartRate + glucose", "binomial", bcols, fcols, intcols, 4)

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



Steps of ECM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...


Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



Steps of ECM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...


Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



Steps of ECM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...


Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



Steps of ECM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...


Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...



In [65]:
pb9 <- PB(sim_imp9, real_coefs)
pb9

[1]   6.7834768 159.1152734   5.0942651   0.9404085  11.0462082

In [66]:
cr9 <- CR(sim_imp9, real_coefs)
cr9

[1] 0.871 0.745 0.894 0.917 0.840

In [67]:
aw9 <- AW(sim_imp9, real_coefs)
aw9

[1] 1.982038737 0.029048635 0.008549221 0.018907645 0.015351481

##### Decision tree imputation

In [68]:
method10 <- rep("", ncol(fram_mcar))
method10[c(11, 12, 14, 15)] <- "cart"

In [69]:
sim_imp10 <- simulate_mice(1000, fram_mcar, 1, 1, method10, "TenYearCHD ~ sysBP + diaBP + heartRate + glucose", "binomial", intcols, 4)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [70]:
pb10 <- PB(sim_imp10, real_coefs)
pb10

[1]  4.869936  1.325733 74.904205 23.276089 10.934707

In [71]:
cr10 <- CR(sim_imp10, real_coefs)
cr10

[1] 0.913 0.911 0.861 0.914 0.852

In [72]:
aw10 <- AW(sim_imp10, real_coefs)
aw10

[1] 1.98798511 0.01396924 0.02681221 0.01875495 0.00833894

##### Random forest imputation

In [73]:
method11 <- rep("", ncol(fram_mcar))
method11[c(11, 12, 14, 15)] <- "rf"

In [74]:
sim_imp11 <- simulate_mice(1000, fram_mcar, 1, 1, method11, "TenYearCHD ~ sysBP + diaBP + heartRate + glucose", "binomial", intcols, 4)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [75]:
pb11 <- PB(sim_imp11, real_coefs)
pb11

[1]  2.086776  7.052881 17.987456 22.207679  7.379741

In [76]:
cr11 <- CR(sim_imp11, real_coefs)
cr11

[1] 0.931 0.903 0.917 0.916 0.876

In [77]:
aw11 <- AW(sim_imp11, real_coefs)
aw11

[1] 2.003862903 0.013301512 0.025696800 0.018789741 0.008403567

##### Summary

In [84]:
pbs <- rbind(c("mean", pb1), c("linear regression", pb2), c("stochastic regression", pb3), c("pmm", pb4), c("MICE with linreg", pb5), 
c("MICE with stochreg", pb6), c("MICE with pmm", pb7), c("KNN", pb8), c("MLMI", pb9), c("decision tree", pb10), c("random forest", pb11))
pbs <- as.data.frame(pbs)
colnames(pbs)[2:6] <- names(real_coefs)
pbs[,2:6] <- pbs[,2:6] %>% lapply(as.numeric)
pbs$mean_pb <- rowMeans(pbs[,2:6])
pbs <- subset(pbs, select=-c(2))
pbs

,V1,sysBP,diaBP,heartRate,glucose,mean_pb
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,mean,14.823165,127.288216,19.48215207,1.3868037,33.21278
2,linear regression,25.829963,280.898691,24.06769194,2.5248557,68.34547
3,stochastic regression,5.816297,105.753590,3.24791507,1.7368696,24.29171
4,pmm,5.982593,111.405156,1.68945146,5.8931503,26.25351
5,MICE with linreg,24.999606,292.354424,17.15386049,0.5741582,68.99600
6,MICE with stochreg,10.203865,150.241152,0.04157387,5.3450018,34.51498
7,MICE with pmm,10.185672,162.040930,9.12608619,6.9160903,39.07829
8,KNN,1.792562,79.223039,29.91745112,9.5409725,25.40714
9,MLMI,159.115273,5.094265,0.94040855,11.0462082,36.59593


In [85]:
crs <- rbind(c("mean", cr1), c("linear regression", cr2), c("stochastic regression", cr3), c("pmm", cr4), c("MICE with linreg", cr5), 
c("MICE with stochreg", cr6), c("MICE with pmm", cr7), c("KNN", cr8), c("MLMI", cr9), c("decision tree", cr10), c("random forest", cr11))
crs <- as.data.frame(crs)
colnames(crs)[2:6] <- names(real_coefs)
crs[,2:6] <- crs[,2:6] %>% lapply(as.numeric)
crs$mean_cr <- rowMeans(crs[,2:6])
crs <- subset(crs, select=-c(2))
crs

,V1,sysBP,diaBP,heartRate,glucose,mean_cr
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,mean,0.760,0.828,0.948,0.923,0.8782
2,linear regression,0.628,0.559,0.882,0.874,0.7558
3,stochastic regression,0.872,0.799,0.900,0.911,0.8728
4,pmm,0.857,0.776,0.902,0.859,0.8518
5,MICE with linreg,0.655,0.563,0.885,0.881,0.7584
6,MICE with stochreg,0.895,0.819,0.943,0.916,0.8928
7,MICE with pmm,0.903,0.833,0.944,0.916,0.8992
8,KNN,0.936,0.879,0.914,0.874,0.8988
9,MLMI,0.745,0.894,0.917,0.840,0.8534


In [86]:
aws <- rbind(c("mean", aw1), c("linear regression", aw2), c("stochastic regression", aw3), c("pmm", aw4), c("MICE with linreg", aw5), 
c("MICE with stochreg", aw6), c("MICE with pmm", aw7), c("KNN", aw8), c("MLMI", aw9), c("decision tree", aw10), c("random forest", aw11))
aws <- as.data.frame(aws)
colnames(aws)[2:6] <- names(real_coefs)
aws[,2:6] <- aws[,2:6] %>% lapply(as.numeric)
aws$mean_aw <- rowMeans(aws[,2:6])
aws <- subset(aws, select=-c(2))
aws

,V1,sysBP,diaBP,heartRate,glucose,mean_aw
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,mean,0.01223440,0.025897099,0.02039089,0.009060381,0.4679119
2,linear regression,0.01683675,0.032695826,0.02068551,0.009117144,0.4352301
3,stochastic regression,0.01464321,0.027858616,0.01884119,0.008553207,0.4117148
4,pmm,0.01419819,0.027173546,0.01887511,0.008487076,0.4117786
5,MICE with linreg,0.01744682,0.033955778,0.02070641,0.009018440,0.4330490
6,MICE with stochreg,0.01734282,0.033734248,0.02095992,0.009004434,0.4358099
7,MICE with pmm,0.01796394,0.035824549,0.02163111,0.009586391,0.4499189
8,KNN,0.01463324,0.028824362,0.01927754,0.008473436,0.4227238
9,MLMI,0.02904864,0.008549221,0.01890765,0.015351481,0.4107791


In [ ]:
+